In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split

import keras
from keras import Sequential
from keras.layers import Dense,Dropout
from keras import backend as K
from keras.constraints import Constraint

import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [2]:
# read data
t = pd.read_csv('../input/lish-moa/train_features.csv')
tt = pd.read_csv('../input/lish-moa/train_targets_scored.csv')



# pre processing
tt = tt[t["cp_type"]!="ctl_vehicle"]
t = t[t["cp_type"]!="ctl_vehicle"]
t["dose_enc"] = 0
t.loc[t["cp_dose"]=="D1","dose_enc"] = 1


# feature vector column names
n_g = 772
n_c = 100

g_cols = ["g-"+str(i) for i in range(n_g)]
c_cols = ["c-"+str(i) for i in range(n_c)]
cat_cols = ["dose_enc","cp_time"]

tt_cols = list(tt.columns)
tt_cols.remove('sig_id')



# train and test data generation
X = t[g_cols+c_cols+cat_cols]
y = tt[tt_cols]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [3]:
model = Sequential()

model.add(Dense(200, input_dim=874, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(206, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode="min", restore_best_weights=True)]

model.fit(X_train,y_train,
          validation_data=(X_test, y_test),
          callbacks=my_callbacks,
          epochs=100)

print(model.evaluate(X_train,y_train))
print(model.evaluate(X_test,y_test))

Epoch 1/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0464 - accuracy: 0.0353 - val_loss: 0.0223 - val_accuracy: 0.0601
Epoch 2/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0231 - accuracy: 0.0669 - val_loss: 0.0223 - val_accuracy: 0.0879
Epoch 3/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0216 - accuracy: 0.0834 - val_loss: 0.0210 - val_accuracy: 0.1014
Epoch 4/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0204 - accuracy: 0.0949 - val_loss: 0.0202 - val_accuracy: 0.1005
Epoch 5/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0196 - accuracy: 0.0955 - val_loss: 0.0204 - val_accuracy: 0.1114
Epoch 6/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0189 - accuracy: 0.1012 - val_loss: 0.0189 - val_accuracy: 0.1043
Epoch 7/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0183 - accuracy: 0.1050 - val_loss: 0.0186 - val_accuracy: 0.1103

In [4]:
class Sparse(Constraint):
    '''
    weight masking to induce sparsity
    '''
    
    def __init__(self, mask):
        self.mask = K.cast_to_floatx(mask)
    
    def __call__(self,x):
        return self.mask * x
    
    def get_config(self):
        return {'mask': self.mask}
    

class WeightsInitialiser(tf.keras.initializers.Initializer):
    '''
    Intialize weights with custom matrices
    '''

    def __init__(self, weights):
      self.weights = weights

    def __call__(self, shape, dtype=None):
      return self.weights

    def get_config(self):  # To support serialization
      return {'weights': self.weights}


def create_sparsity_masks(model,sparsity):
    weights_list = model.get_weights()
    masks = []
    for i,sparse_val in enumerate(sparsity):
        weights = weights_list[2*i]
        #We can ignore biases
        if len(weights.shape) > 1:
            weights_abs = np.abs(weights)
            masks.append((weights_abs>np.percentile(weights_abs,sparse_val))*1.)
    return masks

masks = create_sparsity_masks(model,[95,75,50])

In [5]:
prev_model_weights = model.get_weights()

sparse_model = Sequential()

sparse_model.add(Dense(200, input_dim=874, activation='relu', kernel_constraint=Sparse(masks[0]), 
                       kernel_initializer=WeightsInitialiser(prev_model_weights[0]),
                       bias_initializer=WeightsInitialiser(prev_model_weights[1])))
sparse_model.add(keras.layers.Dropout(0.5))

sparse_model.add(Dense(100, activation='relu', kernel_constraint=Sparse(masks[1]), 
                       kernel_initializer=WeightsInitialiser(prev_model_weights[2]),
                       bias_initializer=WeightsInitialiser(prev_model_weights[3])))
sparse_model.add(keras.layers.Dropout(0.5))

# sparse_model.add(Dense(50, activation='relu',# kernel_constraint=Sparse(masks[2]), 
#                        kernel_initializer=WeightsInitialiser(prev_model_weights[4]), 
#                        bias_initializer=WeightsInitialiser(prev_model_weights[5])))
                 
sparse_model.add(Dense(100, activation='relu', 
                       kernel_initializer=WeightsInitialiser(prev_model_weights[4]), 
                       bias_initializer=WeightsInitialiser(prev_model_weights[5])))

sparse_model.add(Dense(206, activation='sigmoid', 
                       kernel_initializer=WeightsInitialiser(prev_model_weights[6]),
                       bias_initializer=WeightsInitialiser(prev_model_weights[7])))

sparse_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode="min", restore_best_weights=True)]

sparse_model.fit(X_train,y_train,
          validation_data=(X_test, y_test),
          callbacks=my_callbacks,
          epochs=100)

print(sparse_model.evaluate(X_train,y_train))
print(sparse_model.evaluate(X_test,y_test))

Epoch 1/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0181 - accuracy: 0.0968 - val_loss: 0.0175 - val_accuracy: 0.1098
Epoch 2/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0175 - accuracy: 0.1076 - val_loss: 0.0173 - val_accuracy: 0.1191
Epoch 3/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0172 - accuracy: 0.1080 - val_loss: 0.0172 - val_accuracy: 0.1205
Epoch 4/100
549/549 [==============================] - 2s 4ms/step - loss: 0.0171 - accuracy: 0.1111 - val_loss: 0.0171 - val_accuracy: 0.1185
Epoch 5/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0170 - accuracy: 0.1168 - val_loss: 0.0171 - val_accuracy: 0.1253
Epoch 6/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0169 - accuracy: 0.1181 - val_loss: 0.0171 - val_accuracy: 0.1226
Epoch 7/100
549/549 [==============================] - 2s 3ms/step - loss: 0.0168 - accuracy: 0.1180 - val_loss: 0.0170 - val_accuracy: 0.1235

In [6]:
s = pd.read_csv('../input/lish-moa/test_features.csv')
s["dose_enc"] = 0
s.loc[s["cp_dose"]=="D1","dose_enc"] = 1

sX = s[g_cols+c_cols+cat_cols]
sy = sparse_model.predict(sX)

st = pd.DataFrame(data=sy, columns=y.columns)
st["sig_id"] = s["sig_id"]

st.loc[s["cp_type"]=="ctl_vehicle",y.columns] = 0

st.to_csv("submission.csv", index=False)

In [7]:
print(sparse_model.evaluate(X_train,y_train))
print(sparse_model.evaluate(X_test,y_test))

549/549 [==============================] - 1s 2ms/step - loss: 0.0148 - accuracy: 0.1640
[0.014812911860644817, 0.16402779519557953]
138/138 [==============================] - 0s 2ms/step - loss: 0.0166 - accuracy: 0.1337
[0.016646960750222206, 0.1337129771709442]
